In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd

In [3]:
model = YOLO('yolov8n-pose.pt')  # 경량 포즈 모델

results = model.predict(source='0002_jpg.rf.0782a3f913023196e4780c17003b51a9.jpg', save=False, conf=0.5)

# keypoint 추출
keypoints = results[0].keypoints.xy[0].cpu().numpy()  # (17, 2)
print(keypoints)


image 1/1 c:\VisualStudio-WorkSpace\conetComputerVision\personalproject\0002_jpg.rf.0782a3f913023196e4780c17003b51a9.jpg: 640x640 1 person, 87.4ms
Speed: 2.4ms preprocess, 87.4ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
[[          0           0]
 [          0           0]
 [          0           0]
 [          0           0]
 [     120.19      5.4392]
 [     87.694      73.061]
 [     87.073      76.786]
 [      122.8      234.78]
 [     119.79       246.9]
 [     242.28      213.44]
 [     248.52      214.64]
 [     81.813      330.96]
 [     82.822      344.46]
 [     265.15      404.16]
 [     276.06      417.58]
 [          0           0]
 [          0           0]]


In [5]:
import os
import cv2
import pandas as pd
from ultralytics import YOLO

# === 설정 ===
SPLITS = ['train', 'valid', 'test']
BASE_DIR = 'roboflow'
OUTPUT_DIR = 'output'
MODEL_PATH = 'yolo_weights/yolov8m-pose.pt'

CLASS_MAP = {
    0: 'bad',
    1: 'good'
}

model = YOLO(MODEL_PATH)

def get_label_from_txt(txt_path):
    if not os.path.exists(txt_path):
        return None
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        if not lines:
            return None
        class_idx = int(lines[0].split()[0])
        return CLASS_MAP.get(class_idx)

# === 각 split별로 처리 ===
for split in SPLITS:
    image_dir = os.path.join(BASE_DIR, split, 'images')
    label_dir = os.path.join(BASE_DIR, split, 'labels')
    output_csv = os.path.join(OUTPUT_DIR, f'{split}_keypoints.csv')

    rows = []

    for file_name in os.listdir(image_dir):
        if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        txt_name = file_name.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(label_dir, txt_name)
        label = get_label_from_txt(label_path)

        if label is None:
            print(f'[❌] 라벨 없음 또는 라벨 파일 문제: {file_name}')
            continue

        image_path = os.path.join(image_dir, file_name)
        image = cv2.imread(image_path)

        results = model(image)
        keypoints = results[0].keypoints

        if keypoints is None or len(keypoints.data) == 0:
            print(f'[❌] Keypoint 없음: {file_name}')
            continue

        kp = keypoints.data[0][:, :2].cpu().numpy().flatten()
        row = list(kp) + [label]
        rows.append(row)

    if not rows:
        print(f'[⚠️] {split} split에서 유효한 데이터 없음.')
        continue

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    columns = [f'x{i}' if i % 2 == 0 else f'y{i//2}' for i in range(34)] + ['label']
    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f'[✅] {split} → {output_csv} 저장 완료.')


100%|██████████| 50.8M/50.8M [00:01<00:00, 31.5MB/s]



0: 640x640 1 person, 375.5ms
Speed: 2.8ms preprocess, 375.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 319.6ms
Speed: 2.8ms preprocess, 319.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 330.4ms
Speed: 2.2ms preprocess, 330.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 328.0ms
Speed: 2.4ms preprocess, 328.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 295.6ms
Speed: 2.5ms preprocess, 295.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 307.3ms
Speed: 2.0ms preprocess, 307.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 320.4ms
Speed: 2.4ms preprocess, 320.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 302.2ms
Speed: 2.2ms preprocess, 302.2ms inference, 1.2ms postprocess per image at